### Conversor de shp para json para utilizar a aoi local

In [134]:
#add

### Importação das bibliotecas e autenticação da GEE API

In [135]:
import ee
import geemap

try:
    ee.Initialize(project = 'teste-476811')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'teste-476811')

### Criação do mapa interativo e área de interesse

In [136]:
Map = geemap.Map()
aoi = ee.FeatureCollection('projects/teste-476811/assets/Para')

### Máscara de nuvens e fatores de escala

In [137]:
# Funções de máscara para nuvens e saturação de pixels
def cloudmask(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(1 << 3).eq(0)  # Nuvens
    saturationMask = image.select('QA_RADSAT').eq(0)  # Saturação
    return image.updateMask(qaMask).updateMask(saturationMask)

def scalefactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

### Criação da função de Indiciamento

In [138]:
def getNBR(image):
    nbr = image.normalizedDifference(['SR_B4', 'SR_B5']).rename('NBR')
    return image.addBands(nbr)

### Criação do dataset bruto e tratado

In [139]:
# Função para escolher coleção correta por ano
def landsat_collection_by_year(year):
    if (year < 1999) or year in [2003, 2004]: # Note que pegaremos o Landsat 5 para 2003 e 2004, pois o Landsat 7 estava com problemas técnicos
        return 'LANDSAT/LT05/C02/T1_L2'   # Landsat 5
    else:
        return 'LANDSAT/LE07/C02/T1_L2'   # Landsat 7

# Função para tratamento da coleção
def collection_treatment(collection):
    return (collection
            .map(cloudmask)
            .map(scalefactors)
            .median()
            .clip(aoi)
           )

# Cria coleções de x a y automaticamente, podendo escolher se queremos tratamento ou não e se recebem indiciamento ou não
def create_datasets(dataset, start_year, end_year, treatment=False, indexation=False):
    for year in range(start_year, end_year + 1):
        collection_id = landsat_collection_by_year(year)
        median_image = ee.ImageCollection(collection_id).filterDate(f'{year}-01-01', f'{year}-12-31')

        if treatment and indexation:
            dataset[str(year)] = getNBR(collection_treatment(median_image))
            continue
        
        if treatment:
            dataset[str(year)] = collection_treatment(median_image)
            continue
        
        if indexation:
            dataset[str(year)] = getNBR(median_image)
            continue
        
        dataset[str(year)] = median_image
    return dataset

dataset_cloudy = create_datasets({}, 1988, 2007)
dataset_treated = create_datasets({}, 1988, 2007, treatment=True)
dataset_final = create_datasets({}, 1988, 2007, treatment=True, indexation=True)

### Exemplo de visualização do mosaico da mediana tratado (verificação de qualidade da máscara e fator de escala)

In [140]:
ano = 1988

raw = dataset_cloudy[str(ano)].median().clip(aoi)
raw_visualization_params = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0,
    'max': 30000,
    'gamma': 1.4,
}

treated = dataset_treated[str(ano)]
treated_visualization_params = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0,
    'max': 0.3,
    'gamma': 1.4,
}

coord = {
    'lat' : aoi.geometry().centroid().coordinates().get(1).getInfo(), 
    'long' : aoi.geometry().centroid().coordinates().get(0).getInfo()
}

print(f'Centroide da AOI - Latitude: {coord['lat']}, Longitude: {coord['long']}')

Map.addLayer(raw, raw_visualization_params, f'{ano} - Cloudy')
Map.addLayer(treated, treated_visualization_params, f'{ano} - Treated')
Map.addLayer(aoi.style(color = 'red', fillColor = '00000000'), {}, 'AOI')
Map.setCenter(coord['long'], coord['lat'], 6)
Map.set_zoom(6)
Map

Centroide da AOI - Latitude: -3.974512955502294, Longitude: -53.07056416496682


Map(center=[-3.974512955502294, -53.07056416496682], controls=(WidgetControl(options=['position', 'transparent…

### Visualização das imagens no mapa

In [141]:
# Divisão do dataset em intervalos de 5 anos para a avaliação do desmatamento em cada período através do indiciamento NBR
intervals = [
    (1988, 1992),
    (1993, 1997),
    (1998, 2002),
    (2003, 2007)
]

# Cálculo do Delta NBR para cada intervalo e visualização no mapa
for start_year, end_year in intervals:
    nbr_start = dataset_final[str(start_year)].select('NBR')
    nbr_end = dataset_final[str(end_year)].select('NBR')
    delta_nbr = nbr_start.subtract(nbr_end).rename(f'Delta_NBR_{start_year}_{end_year}')
    nbr_visualization_params = {
        'min': -0.5, 
        'max': 0.5, 
        'palette': ['blue', 'white', 'red']
        }
    Map.addLayer(delta_nbr.clip(aoi), nbr_visualization_params, f'Delta NBR {start_year}-{end_year}')

# Visualização do desmatamento total no período analisado
total_nbr_start = dataset_final['1988'].select('NBR') 

Map.centerObject(aoi, 6)
Map

Map(center=[-3.974512955502294, -53.07056416496682], controls=(WidgetControl(options=['position', 'transparent…